In [535]:
import pandas as pd
from wikidata.client import Client
import wikipedia
import requests 
import regex as re

In [27]:
def fetch_wikidata(params):
    url = 'https://www.wikidata.org/w/api.php'
    try:
        return requests.get(url, params=params)
    except:
        return 'There was and error'

In [102]:
# What text to search for
query = 'Chicago Sun-Times'
 
# Which parameters to use
params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': query,
        'language': 'en'
    }

# Fetch API
data = fetch_wikidata(params)
 

In [104]:
data.json()["search"]

[{'id': 'Q577417',
  'title': 'Q577417',
  'pageid': 543081,
  'display': {'label': {'value': 'Chicago Sun-Times', 'language': 'en'},
   'description': {'value': 'Chicago daily newspaper', 'language': 'en'}},
  'repository': 'wikidata',
  'url': '//www.wikidata.org/wiki/Q577417',
  'concepturi': 'http://www.wikidata.org/entity/Q577417',
  'label': 'Chicago Sun-Times',
  'description': 'Chicago daily newspaper',
  'match': {'type': 'label', 'language': 'en', 'text': 'Chicago Sun-Times'}}]

In [48]:
#show response as JSON
firstResult = data.json()["search"][0]
entity = firstResult["id"]

In [49]:
entityList = [entity]

In [55]:
params = {
    'action': 'wbgetentities',
    'ids':"|".join(entityList),
    'format': 'json',
    'languages': 'en'
}

data = fetch_wikidata(params)

In [63]:
data.json()["entities"][entityList[0]].keys()

dict_keys(['pageid', 'ns', 'title', 'lastrevid', 'modified', 'type', 'id', 'labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [96]:
data.json()["entities"][entityList[0]]["claims"].keys()

dict_keys(['P373', 'P407', 'P856', 'P646', 'P1042', 'P236', 'P31', 'P1476', 'P127', 'P571', 'P123', 'P495', 'P3417', 'P154', 'P3912', 'P18', 'P1417', 'P291', 'P5454', 'P17', 'P131', 'P2013', 'P2002', 'P6136', 'P1258', 'P6295', 'P6379', 'P159', 'P2003', 'P9035', 'P7363', 'P2390', 'P7901', 'P3393', 'P9852', 'P10006', 'P910', 'P2088'])

In [337]:
outletDf = pd.read_csv("/shared/3/projects/newsDiffusion/data/raw/NELAdata/CSVs/outlets.csv")

In [338]:
prettyNamesDf = pd.read_csv("/shared/3/projects/newsDiffusion/data/processed/newsData/formattedLocalNames.csv", names=["prettyNames", "junk"], quotechar="'").drop(columns=["junk"])

In [339]:
import regex 

In [340]:
#clean up strings a bit 
prettyNamesDf["prettyNames"] = prettyNamesDf["prettyNames"].apply(lambda x: regex.sub(r"[^A-Za-z ]", "", x).strip())

In [483]:
prettyNamesDf["ogNames"] = outletDf["source"]

In [332]:
from tqdm import tqdm

In [126]:
outletDf[outletDf["source"] == "dailynews"]

,sourcedomain_id,fips,source,description,onlinesince,rank,state,city,url,lon,lat,county
23,dailynews-dailynews.com,6037,dailynews,Los Angeles based newspaper with online and pr...,22-Feb-1999,31515.0,California,Woodland Hills,NaN,-118.608975,34.165357,Los Angeles


In [498]:
prettyNamesDf

,prettyNames,ogNames
0,andalusia star news,andalusiastarnews
1,atmore advance,atmoreadvance
2,the brewton standard,thebrewtonstandard
3,clanton advertiser,clantonadvertiser
4,gadsden messenger,gadsdenmessenger
...,...,...
308,southwashington county bulletin,southwashingtoncountybulletin
309,standard press,standardpress
310,massachusetts news,massachusettsnews
311,new hampshire gazette,newhampshiregazette


## Find Article Titles / Search Results

In [488]:
searchResults = []

#just get the title of the article that we are looking for
for i, nameTup in tqdm(prettyNamesDf[["prettyNames"]].itertuples()): 
    ogName = nameTup[0]
    pName = nameTup[1]
    
    # Which parameters to use
    # pName is the outlet we are searching for 
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': pName,
        'language': 'en'
    }
    
    # Fetch API
    data = fetch_wikidata(params).json()
    searchResults.append((ogName, pName, data["search"]))

313it [01:27,  3.57it/s]


In [489]:
firstResults = [(ogName, pName, searchResult[0]) for ogName, pName, searchResult in searchResults if len(searchResult) > 0]
entityList =  [searchResult[0]["id"] for ogName, pName, searchResult in searchResults if len(searchResult) > 0]

## Extract Infoboxes from Articles

In [491]:
entityData = []
for ogName, pName, result in tqdm(firstResults): 
    params = {
        'action': 'wbgetentities',
        'ids':result["id"],
        'format': 'json',
        'languages': 'en'
    }

    entityData.append((ogName, pName, fetch_wikidata(params).json()))

100%|██████████| 313/313 [00:48<00:00,  6.44it/s]


In [492]:
entityData[0][1]["entities"].keys()

TypeError: string indices must be integers

In [209]:
allPropTups = []
#get all of the properties and values in our search results 
for i, entityTup in enumerate(entityData):
    entity = entityList[i]
    pName = entityTup[0]
    entJson = entityTup[1]
    
    #here we have the dict with properties and their values 
    prop = entJson["entities"][entity]["claims"]
    
    propTups = [(pName, entKey, entDict[0]["id"]) for entKey, entDict in entJson["entities"][entity]["claims"].items()]
    allPropTups += propTups

In [280]:
#entityData[0][1]["entities"]['Q55642880']["claims"]

In [281]:
for key in entityData[0][1]["entities"]['Q55642880']["claims"]: 
    for subEntList in entityData[0][1]["entities"]['Q55642880']["claims"][key]: 
        
        for subEnt in subEntList: 
            #print(subEntList[subEnt])
            #print("-------------------------__")
            pass

In [215]:
propDf = pd.DataFrame(allPropTups, columns=["outlet", "property", "entityId"])

In [231]:
propDf["entityId"] = propDf["entityId"].apply(lambda x: x.split("$")[0])

In [245]:
ownership = propDf[(propDf["property"] == "P127") | (propDf["property"] == "P749")]

In [246]:
ownership.groupby("entityId").agg(list)

,outlet,property
entityId,,
Q1056889,[ deseret news],[P127]
Q1067299,[ the star],[P127]
Q107087592,[ detroit lakes tribune],[P127]
Q13447162,[ daily democrat],[P127]
Q13447289,[ estes park trail gazette],[P127]
...,...,...
Q7981780,[ wellsville daily reporter],[P127]
Q7984692,[ west central tribune],[P127]
Q7985331,[ west hawaii today],[P127]


In [251]:
entity = client.get('Q1056889', load=True).lists()

In [289]:
#TODO: try this approach of extracting the infoboxes directly 
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/Estes_Park_Trail-Gazette'
infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
#wikitables = read_html(page, index_col=0, attrs={"class":"wikitable"})

print("Extracted {num} infoboxes".format(num=len(infoboxes)))
#print("Extracted {num} wikitables".format(num=len(wikitables)))
     

Extracted 1 infoboxes


In [287]:
infoboxes[0]

,1
0,
Type,Weekly newspaper
Format,Broadsheet
Owner(s),Prairie Mountain Publishing (MediaNews Group)
Publisher,Michael Romero
Editor,Patti Brown
Founded,1912
Headquarters,"251 Moraine Avenue Estes Park, CO 80517 United..."
Website,EPTrail.com


In [506]:
for i, nameTup in tqdm(prettyNamesDf.iterrows()):
    pass

313it [00:00, 12518.64it/s]


In [550]:
ownerTups = []
for i, nameTup in tqdm(prettyNamesDf.iterrows()):
    ogName = nameTup["ogNames"]
    pName = nameTup["prettyNames"]
    result = wikipedia.search(pName, results = 1)
    try: 
        page = wikipedia.page(result[0]).url
        infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
        df = pd.DataFrame(infoboxes[0]).reset_index()
        df.columns = ["field", "value"]
        df["field"] = df["field"].fillna("")
        owner = list(df.loc[df["field"].str.contains("Owner"), "value"])
        if len(owner) > 0:
            ownerTups.append((ogName, pName, result[0], owner[0]))
        else: 
            ownerTups.append((ogName, pName, result[0], None))
    except: 
        ownerTups.append((ogName, pName, "", ""))

18it [00:10,  1.32it/s]/opt/anaconda/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
23it [00:13,  1.53it/s]/opt/anaconda/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual enviro

In [551]:
ownerTups[0]

('andalusiastarnews',
 'andalusia star news',
 'Andalusia Star News',
 'Boone Newspapers')

In [593]:
ownershipDf = pd.DataFrame(ownerTups)
ownershipDf.columns = ["ogName", "pName", "search result", "owner"]

## Ownership from UNC Source

In [585]:
#get ownership information from UNC source
UNCdf = pd.read_csv("/shared/3/projects/newsDiffusion/data/raw/UNCOwnership/UNCNewspaperDatabase_12_17_20.csv")
UNCdf.head()

,state,newspaper_id,newspaper_name,frequency,city,county,owner_id,owner_name,owner_type,total_circulation,days_published
0,AK,50003,Anchorage Press,W,Anchorage,Anchorage Municipality,5813.0,Wick Communications,Private,20000.0,1
1,AK,18776,The Bristol Bay Times/Dutch Harbor Fisherman,W,Anchorage,Anchorage Municipality,6028.0,Ryan Binkley & Jason Evans,Private,NaN,1
2,AK,18654,The Arctic Sounder,W,Anchorage,Anchorage Municipality,6028.0,Ryan Binkley & Jason Evans,Private,2000.0,1
3,AK,10092,Anchorage Daily News,D,Anchorage,Anchorage Municipality,6028.0,Ryan Binkley & Jason Evans,Private,33301.0,7
4,AK,18943,The Cordova Times,W,Cordova,Valdez-Cordova,6045.0,Native Village of Eyak,Private,969.0,1


In [586]:
#remove anything that isn't a number or letter 
def cleanStr(inStr): 
    inStr = inStr.lower()
    return re.sub("[^a-z0-9]", "", inStr)

UNCdf["ogName"] = UNCdf["newspaper_name"].apply(cleanStr)

In [587]:
len(ownerTups)

313

In [588]:
len(ownershipDf)

313

In [594]:
ownershipDf = pd.merge(ownershipDf, UNCdf[["ogName", "owner_name"]], on="ogName", how="left")
ownershipDf = ownershipDf.rename(columns={"owner_name":"UNCOwner", "owner":"WikiOwner"})

In [595]:
ownershipDf[["UNCOwner", "WikiOwner"]] = ownershipDf[["UNCOwner", "WikiOwner"]].fillna("")
ownershipDf = ownershipDf[(ownershipDf["UNCOwner"] != "") | (ownershipDf["WikiOwner"] != "")]

In [597]:
len(ownershipDf)

250

In [609]:
mergedOwner = []
for i, wikiOwner, UNCOwner in ownershipDf[["WikiOwner", "UNCOwner"]].itertuples():
    if len(wikiOwner) > 0: 
        mergedOwner.append(wikiOwner)
    else: 
        mergedOwner.append(UNCOwner)
        
ownershipDf["mergedOwner"] = mergedOwner

In [610]:
ownershipDf

,ogName,pName,search result,WikiOwner,UNCOwner,mergedOwner
0,andalusiastarnews,andalusia star news,Andalusia Star News,Boone Newspapers,Boone Newspapers,Boone Newspapers
1,atmoreadvance,atmore advance,Atmore Advance,Morris Newspaper Corporation/Boone Newspapers ...,Boone Newspapers,Morris Newspaper Corporation/Boone Newspapers ...
4,gadsdenmessenger,gadsden messenger,,,"McCharthy, Chris","McCharthy, Chris"
5,greenvilleadvocate,greenville advocate,"Greenville, Illinois",,Endress Jay & Paula,Endress Jay & Paula
6,selmatimesjournal,selma times journal,Selma Times-Journal,Boone Newspapers Inc.,,Boone Newspapers Inc.
...,...,...,...,...,...,...
330,huntingtonnews,huntington news,The Huntington News,World Series Way Publishing Company Inc.,,World Series Way Publishing Company Inc.
331,themorganmessenger,the morgan messenger,Morgan Messenger,"The Morgan Messenger, Inc.",Morgan Messenger Inc The,"The Morgan Messenger, Inc."
334,thedailyreporter,the daily reporter,Wellsville Daily Reporter,Gannett,,Gannett
336,southwashingtoncountybulletin,southwashington county bulletin,,,Forum Communications,Forum Communications


In [612]:
ownershipDf[["mergedOwner", "pName"]].groupby("mergedOwner").agg(len).sort_values("pName", ascending=False).rename(columns={"pName":"outletCount"})

,outletCount
mergedOwner,
Gannett,22
Gannett/Gatehouse,12
Digital First Media,10
Sound Publishing,9
Boone Newspapers,9
...,...
Forum Communications Company,1
"Fobes, Jeff",1
Endress Jay & Paula,1


In [615]:
ownershipDf["cleanedOwner"] = ownershipDf["mergedOwner"]

In [651]:
fuzzTups = []
#now see if we can do any fuzzy matching on owners 
for owner1 in ownershipDf["mergedOwner"]: 
    for owner2 in ownershipDf["mergedOwner"]: 
        fuzzTups.append((owner1, owner2, fuzz.ratio(owner1, owner2)))

matchDf = pd.DataFrame(fuzzTups, columns=["owner1", "owner2", "fuzzRatio"])

In [654]:
matches = matchDf[(matchDf["fuzzRatio"] > 80) & (matchDf["fuzzRatio"] < 100)].drop_duplicates()
matches

,owner1,owner2,fuzzRatio
4,Boone Newspapers,Boone Newspapers Inc.,86
177,Boone Newspapers,"Boone Newspapers, Inc.",84
1000,Boone Newspapers Inc.,Boone Newspapers,86
1177,Boone Newspapers Inc.,"Boone Newspapers, Inc.",98
1183,Boone Newspapers Inc.,Ogden Newspapers Inc.,81
1403,Sound Publishing,Sunrise Publishing,82
3540,Lewis Newspapers,Swift Newspapers,81
4311,JAMS Media,AIM Media,84
7433,Ogden Newspapers,Ogden Newspapers Inc.,86
10014,Swift Newspapers,Lewis Newspapers,81


In [655]:
#gatehouse is the successor 
replaceTups = [("Gannett", "Gannett"), ("GateHouse", "Gannett"), ("Boone", "Boone Newspapers"), ("AIM Media", "AIM Media"), ("MediaNews", "Digital First"), 
               ("Cherry", "Cherry Road Media"), ("Ogden", "Ogden Newspapers"), ("Community Newspapers", "Community Newspapers"), ("Specht Newspapers", "Specht Newspapers"),
              ("MaineToday", "MaineToday"), ("Forum Communications", "Forum Communications")]

for keyword, replaceWord in replaceTups: 
    ownershipDf.loc[ownershipDf["mergedOwner"].str.contains(keyword),"cleanedOwner"] = replaceWord


In [656]:
longTail = ownershipDf[["cleanedOwner", "pName"]].groupby("cleanedOwner").agg(len).sort_values("pName", ascending=False).rename(columns={"pName":"outletCount"})

#longTail.loc[longTail["outletCount"] == 1].index

In [657]:
longTail = ownershipDf[["cleanedOwner", "pName"]].groupby("cleanedOwner").agg(len).sort_values("pName", ascending=True).rename(columns={"pName":"outletCount"})

In [663]:
ownershipDf = ownershipDf.drop_duplicates(["ogName"])

In [665]:
ownershipDf

,ogName,pName,search result,WikiOwner,UNCOwner,mergedOwner,cleanedOwner
0,andalusiastarnews,andalusia star news,Andalusia Star News,Boone Newspapers,Boone Newspapers,Boone Newspapers,Boone Newspapers
1,atmoreadvance,atmore advance,Atmore Advance,Morris Newspaper Corporation/Boone Newspapers ...,Boone Newspapers,Morris Newspaper Corporation/Boone Newspapers ...,Boone Newspapers
4,gadsdenmessenger,gadsden messenger,,,"McCharthy, Chris","McCharthy, Chris","McCharthy, Chris"
5,greenvilleadvocate,greenville advocate,"Greenville, Illinois",,Endress Jay & Paula,Endress Jay & Paula,Endress Jay & Paula
6,selmatimesjournal,selma times journal,Selma Times-Journal,Boone Newspapers Inc.,,Boone Newspapers Inc.,Boone Newspapers
...,...,...,...,...,...,...,...
330,huntingtonnews,huntington news,The Huntington News,World Series Way Publishing Company Inc.,,World Series Way Publishing Company Inc.,World Series Way Publishing Company Inc.
331,themorganmessenger,the morgan messenger,Morgan Messenger,"The Morgan Messenger, Inc.",Morgan Messenger Inc The,"The Morgan Messenger, Inc.","The Morgan Messenger, Inc."
334,thedailyreporter,the daily reporter,Wellsville Daily Reporter,Gannett,,Gannett,Gannett
336,southwashingtoncountybulletin,southwashington county bulletin,,,Forum Communications,Forum Communications,Forum Communications


In [669]:
ownerCounts = ownershipDf[["cleanedOwner", "ogName"]].groupby("cleanedOwner").agg(len).sort_values("ogName", ascending=False)

In [672]:
ownerCounts

,ogName
cleanedOwner,
Gannett,34
Boone Newspapers,13
Forum Communications,10
Digital First Media,10
Sound Publishing,9
...,...
Greene Publishing Inc.,1
Grand Teton News,1
Grand Rabbi Y. A. Korff,1


In [675]:
len(ownershipDf)

222

In [673]:
ownershipDf.to_csv("/shared/3/projects/newsDiffusion/data/processed/ownership/infoboxOwnership.csv")